# Data Preparation: Preprocessing
Este notebook apresenta a etapa de preparação dos dados, contemplando os passos necessários de pré-processamento, úteis para o treinamento do modelo de reranker.

Neste momento, são então realizados:
- Limpeza profunda e normalização textual
- Normalização do score de relevância
- Chunking dos dados textuais

Ao final, os dados processados são salvos no formato `parquet` para utilização nas etapas futuras.

### **Observações quanto à etapa de Data Understanding**
Com base nas análises feitas na etapa anterior, observou-se que o Quati Dataset por si só apresenta poucos dados relevantes para o treinamento de um modelo de reranker neural.

Tendo isso em vista, optou-se por manter o **Quati Dataset como o conjunto de dados que define o problema de negócio** e foi acrescentado uma **nova base de dados suplementar especificamente para o treinamento do reranker**.

A base de dados selecionada foi o **MS MARCO**, um dataset robusto e padrão para treinar rerankers e retrievers no estado da arte. Ele possui uma estrutura semelhante ao Quati Dataset, uma quantidade massiva de dados, apesar de conter apenas textos em inglês.

Desta forma, será importante considerar análises qualitativas cross-linguísticas ao final do projeto para avaliar perfomance do reranker no contexto do domínio desenvolvido.

----

## Importações e instalações

In [ ]:
# Check available CUDA version
# !nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [1]:
# Instalações
!pip install "datasets<4.0.0"
# !pip install -U spacy   # NLP
# !pip install -U 'spacy[cuda12x]'
# !python -m spacy download pt_core_news_md
# !python -m spacy download en_core_web_md

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
# Imports
# Utils
import os
cpu_count = os.cpu_count()
print(f"CPU count: {cpu_count}")

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

# Manipulação de dados
import numpy as np
import pandas as pd
from datasets import load_dataset, load_from_disk

# NLP
import re
# import spacy
import unicodedata
# print(f"Spacy GPU activation: {spacy.require_gpu()}")

# Embeddings
from transformers import AutoTokenizer

CPU count: 2


## Carregamento dos datasets

### Quati Dataset

In [15]:
# Load Quati passages
quati_passages = load_dataset("parquet", data_files="cleaned_passages.parquet")["train"]

quati_passages

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['passage_id', 'passage'],
    num_rows: 1000000
})

In [10]:
# Load Quati topics
quati_topics = load_dataset("parquet", data_files="./cleaned_topics.parquet")["train"]

quati_topics

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['query_id', 'query'],
    num_rows: 200
})

In [9]:
# Load Quati qrels
quati_qrels = load_dataset("parquet", data_files="./cleaned_qrels.parquet")["train"]

quati_qrels

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['query_id', 'passage_id', 'score'],
    num_rows: 1933
})

### MS MARCO

In [3]:
# Load MS MARCO
msmarco = load_dataset("microsoft/ms_marco", "v2.1", split="train")

msmarco

README.md: 0.00B [00:00, ?B/s]

v2.1/validation-00000-of-00001.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

v2.1/train-00000-of-00007.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

v2.1/train-00001-of-00007.parquet:   0%|          | 0.00/240M [00:00<?, ?B/s]

v2.1/train-00002-of-00007.parquet:   0%|          | 0.00/241M [00:00<?, ?B/s]

...

v2.1/test-00000-of-00001.parquet:   0%|          | 0.00/204M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/101093 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/808731 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/101092 [00:00<?, ? examples/s]

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 808731
})

### Verificações básicas no MS MARCO
Tendo em vista que o MS MARCO não terá uma análise exploratória extensa, dado o seu uso suplementar, é necessário ao menos realizar uma verificação básica (textos nulos ou duplicados, tamanho dos textos, distribuição dos scores) dos dados presentes para segurança e rigorosidade qualitativa dos dados.

#### Contagem de valores nulos

In [4]:
# Check for null values in each column
# Function to count null in batches
def count_nulls(batch):
    return {
        col: [x is None or x == "" for x in batch[col]]
        for col in batch
    }

# Null counts in batches
null_flags = msmarco.map(
    count_nulls,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)

# Sum of batches
null_counts = {
    col: int(sum(null_flags[col]))
    for col in null_flags.column_names
}

# Display
for col, count in null_counts.items():
    print(f"Column '{col}': {count} null values")

Map (num_proc=2):   0%|          | 0/808731 [00:00<?, ? examples/s]

Column 'answers': 0 null values
Column 'passages': 0 null values
Column 'query': 0 null values
Column 'query_id': 0 null values
Column 'query_type': 0 null values
Column 'wellFormedAnswers': 0 null values


#### Contagem de queries e documentos

In [5]:
# Count how many unique queries there are
print(f"Number of unique queries: {len(set(msmarco['query_id']))}")

Number of unique queries: 808731


In [6]:
# Count how many unique passages there are
# Function to count passages per query in batches
# Since each query has its dict with documents
def count_passages(batch):
    return {
        "num_passages": [len(p["passage_text"]) for p in batch["passages"]]
    }

# Passages count in batches
passages_counts_flags = msmarco.map(
    count_passages,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)

# Total passages count
total_passages_count = sum(passages_counts_flags["num_passages"])

# Display
print(f"Total number of passages: {total_passages_count}")
print(f"Min number of passages per query: {np.min(passages_counts_flags['num_passages'])}")
print(f"Mean number of passages per query: {np.mean(passages_counts_flags['num_passages'])}")
print(f"Median number of passages per query: {np.median(passages_counts_flags['num_passages'])}")
print(f"Max number of passages per query: {np.max(passages_counts_flags['num_passages'])}")

Map (num_proc=2):   0%|          | 0/808731 [00:00<?, ? examples/s]

Total number of passages: 8069749
Mean number of passages per query: 9.978285734069797
Median number of passages per query: 10.0


#### Tamanho das queries e documentos

In [7]:
# Length of queries
query_lens = np.array([len(q) for q in msmarco["query"]])

print(f"Min query length: {query_lens.min()}")
print(f"Mean query length: {query_lens.mean()}")
print(f"Median query length: {np.median(query_lens)}")
print(f"Max query length: {query_lens.max()}")

Min query length: 5
Mean query length: 35.44781392082163
Median query length: 33.0
Max query length: 429


In [8]:
# Length of passages
passage_lens = []
for passages in msmarco["passages"]:
    passage_lens.extend(len(p) for p in passages["passage_text"])

passage_lens = np.array(passage_lens)

print(f"Min passage length: {passage_lens.min()}")
print(f"Mean passage length: {passage_lens.mean()}")
print(f"Median passage length: {np.median(passage_lens)}")
print(f"Max passage length: {passage_lens.max()}")

Min passage length: 3
Mean passage length: 335.9978042687573
Median passage length: 300.0
Max passage length: 1397


## Pré-processamento

In [ ]:
# Function definition to clean text
# URL_PATTERN = re.compile(r"https?://\S+|www\.\S+")
# EMAIL_PATTERN = re.compile(r"\S+@\S+")

# Function to clean text in batches
# def clean_text(docs: list[str]) -> list[str]:
#     """
#     Realiza a limpeza textual em lote.
#     As limpezas realizadas são:
#         - Normalização do enconding
#         - Remoção de URL e e-mails
#         - Lematização
#         - Transformação para minúsculas
#         - Remoção de espaços e pontuação

#     Params:
#         docs (list[str]): Lista de textos a serem limpos.

#     Returns:
#         list[str]: Lista de textos limpos.
#     """
#     cleaned_docs = []
#     # Apply URL and email removal first
#     pre_cleaned_texts = [
#         EMAIL_PATTERN.sub(" ", URL_PATTERN.sub(" ", unicodedata.normalize("NFKC", doc)))
#         for doc in docs
#     ]

#     # Process pre-cleaned texts in batches using nlp.pipe
#     for doc_obj in nlp.pipe(pre_cleaned_texts, batch_size=1000):
#         tokens = [
#             token.lemma_.lower()
#             for token in doc_obj
#             if not token.is_punct
#             and not token.is_space
#         ]
#         cleaned_docs.append(" ".join(tokens))

#     return cleaned_docs

In [11]:
HTML_TAG = re.compile(r"<[^>]+>")
URL = re.compile(r"https?://\S+|www\.\S+")
EMAIL = re.compile(r"\S+@\S+")
MULTISPACE = re.compile(r"\s+")

def clean_text_fast(text: str) -> str:
    text = unicodedata.normalize("NFKC", text)
    text = text.lower()
    text = HTML_TAG.sub(" ", text)
    text = URL.sub(" ", text)
    text = EMAIL.sub(" ", text)
    text = re.sub(r"[^\w\sáéíóúãõç]", " ", text)
    text = MULTISPACE.sub(" ", text).strip()
    return text

### Quati Dataset

In [ ]:
# Load Portuguese spacy pipeline
# nlp = spacy.load(
#     "pt_core_news_md",
#     disable=["ner", "parser", "attribute_ruler", "textcat"]
# )

In [13]:
# Clean Quati queries
# quati_topics = quati_topics.map(
#     lambda x: {"query": clean_text(x["query"])},
#     batched=True,
#     batch_size=10_000,
#     num_proc=1 # Changed to 1 to avoid CUDARuntimeError with multiprocessing
# )
def clean_quati_topics(batch):
    batch["clean_query"] = [clean_text_fast(q) for q in batch["query"]]
    return batch

quati_topics = quati_topics.map(
    clean_quati_topics,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)

quati_topics[:3]

Map (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

{'query_id': [0, 1, 2],
 'query': ['qual a maior característica da flora brasileira',
  'qual a maior característica da fauna brasileira',
  'por que os países guiana e suriname não são filiados a conmebol'],
 'clean_query': ['qual a maior característica da flora brasileira',
  'qual a maior característica da fauna brasileira',
  'por que os países guiana e suriname não são filiados a conmebol']}

In [16]:
# Clean Quati passages
def clean_quati_passages(batch):
    batch["clean_passage"] = [clean_text_fast(p) for p in batch["passage"]]
    return batch

quati_passages = quati_passages.map(
    clean_quati_passages,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)

quati_passages[:2]

Map (num_proc=2):   0%|          | 0/1000000 [00:00<?, ? examples/s]

{'passage_id': ['clueweb22-pt0000-00-00003_1', 'clueweb22-pt0000-00-00003_2'],
 'passage': ['se você precisar de ajuda visite o website nacional sobre a covid-19 ou ligue para a linha de apoio à covid-19 808 24 24 24 perguntas mais frequentes posso viajar entre sintra e cascais quais são as restrições de viagem em cascais qual o número de telefone de apoio para a covid 19 em cascais preciso utilizar máscara facial no transporte público em cascais a prática do distanciamento social é compulsória em cascais o que eu devo fazer caso apresente sintomas da covid-19 quando chegar em cascais última atualização 25 abr 2022 aplicam-se exceções para detalhes completos european union estamos trabalhando ininterruptamente para lhe trazer as últimas informações de viagem relacionadas à covid-19 esta informação é compilada a partir de fontes oficiais ao melhor de nosso conhecimento está correta de acordo com a última atualização visite avisos de viagem rome2rio para ajuda geral perguntas respostas q

In [17]:
# Normalize Quati scores
# Function to normalize scores
def normalize_quati_score(score):
    return score / 3.0

quati_qrels = quati_qrels.map(
    lambda x: {"score": normalize_quati_score(x["score"])},
    num_proc=cpu_count
)

quati_qrels[:3]

Map (num_proc=2):   0%|          | 0/1933 [00:00<?, ? examples/s]

{'query_id': [1, 1, 1],
 'passage_id': ['clueweb22-pt0000-78-09747_0',
  'clueweb22-pt0000-96-07278_111',
  'clueweb22-pt0001-85-06153_3'],
 'score': [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]}

### MS MARCO

In [ ]:
# Load English spacy pipeline
# nlp = spacy.load(
#     "en_core_web_md",
#     disable=["ner", "parser", "attribute_ruler", "textcat"]
# )

In [18]:
# Clean MS MARCO queries
# msmarco = msmarco.map(
#     lambda x: {"query": clean_text(x["query"])},
#     batched=True,
#     batch_size=10_000,
#     num_proc=1 # Changed to 1 to avoid CUDARuntimeError with multiprocessing
# )
def clean_msmarco_queries(batch):
    batch["clean_query"] = [clean_text_fast(q) for q in batch["query"]]
    return batch

msmarco = msmarco.map(
    clean_msmarco_queries,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)

msmarco["query"][:3]

Map (num_proc=2):   0%|          | 0/808731 [00:00<?, ? examples/s]

[')what was the immediate impact of the success of the manhattan project?',
 '_________ justice is designed to repair the harm to victim, the community and the offender caused by the offender criminal act. question 19 options:',
 'why did stalin want control of eastern europe']

In [20]:
# Clean MS MARCO passages
def clean_msmarco_passages(batch):
    cleaned_passages_for_batch = []
    for passages_dict_for_one_query in batch["passages"]:
        cleaned_passages_for_one_query = [
            clean_text_fast(p_text)
            for p_text in passages_dict_for_one_query["passage_text"]
        ]
        cleaned_passages_for_batch.append(cleaned_passages_for_one_query)

    batch["clean_passages"] = cleaned_passages_for_batch
    return batch


msmarco = msmarco.map(
    clean_msmarco_passages,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)


msmarco["clean_passages"][:3]

Map (num_proc=2):   0%|          | 0/808731 [00:00<?, ? examples/s]

[['the presence of communication amid scientific minds was equally important to the success of the manhattan project as scientific intellect was the only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant hundreds of thousands of innocent lives obliterated',
  'the manhattan project and its atomic bomb helped bring an end to world war ii its legacy of peaceful uses of atomic energy continues to have an impact on history and science',
  'essay on the manhattan project the manhattan project the manhattan project was to see if making an atomic bomb possible the success of this project would forever change the world forever making it known that something this powerful can be manmade',
  'the manhattan project was the name for a project conducted during world war ii to develop the first atomic bomb it refers specifically to the period of the project from 194 2 1946 under the control of the u s army corps of engineers under

## Chunking

In [ ]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    device="cuda"
)

In [ ]:
def chunk_text(text, max_tokens=256, overlap=64):
    embeddings = tokenizer.encode(
        text,
        batch_size=64,
        normalize_embeddings=True,
        show_progress_bar=True
    )

    # tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk))
    return chunks


## Persistência dos dados processados